In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime
import pytz

In [3]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/game/'
appendix = '/feed/live'

year = '202202'
game = 1

print(str(game).zfill(4))

0001


# Scrape for 2020-21

In [20]:

year = '202002'
date_format_game = "%d-%m-%Y"
date_format_birth = "%Y-%m-%d"

# pprint(game['liveData']['boxscore']['teams']['away']['players'])


awaylogs = {}
homelogs = {}


for i in range(868):
# for i in range(2):    
    
    gameURL = baseURL + year + str(i+1).zfill(4) + appendix
    
    print(gameURL)

    game = requests.get(gameURL).json()
    
    away = game['liveData']['boxscore']['teams']['away']['players']
    awayname = game['gameData']['teams']['away']['abbreviation']
    home = game['liveData']['boxscore']['teams']['home']['players']
    homename = game['gameData']['teams']['home']['abbreviation']
    birthday = game['gameData']['players']

    if game['liveData']['linescore']['currentPeriod'] > 3:
        OT = True
    else:
        OT = False

    awayteam = []

    date = game['gameData']['datetime']['dateTime']

    datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

    utc_timestamp = pytz.utc.localize(datetime_obj)
    eastern_timezone = pytz.timezone('US/Eastern')
    eastern_time = utc_timestamp.astimezone(eastern_timezone)

    formatted_date_string = eastern_time.strftime('%d-%m-%Y')
    game_date = datetime.strptime(formatted_date_string, date_format)

    for player in away:
        awayteam.append(player)

    for player in awayteam:
        name = away[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = away[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = away[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = away[player]['position']['abbreviation']
            stats['name'] = name

        awaylogs[player  + '-' + formatted_date_string] = stats
        awaylogs[player  + '-' + formatted_date_string]['position'] = position
        awaylogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        awaylogs[player  + '-' + formatted_date_string]['name'] = name
        awaylogs[player  + '-' + formatted_date_string]['team'] = awayname
        awaylogs[player  + '-' + formatted_date_string]['opponent'] = homename
        awaylogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        awaylogs[player  + '-' + formatted_date_string]['id'] = player
        awaylogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days


    hometeam = []

    # date = game['gameData']['datetime']['dateTime']

    for player in home:
        hometeam.append(player)

    for player in hometeam:
        name = home[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = home[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = home[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = home[player]['position']['abbreviation']
            stats['name'] = name
            
                

        homelogs[player  + '-' + formatted_date_string] = stats
        homelogs[player  + '-' + formatted_date_string]['position'] = position
        homelogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        homelogs[player  + '-' + formatted_date_string]['name'] = name
        homelogs[player  + '-' + formatted_date_string]['team'] = homename
        homelogs[player  + '-' + formatted_date_string]['opponent'] = awayname
        homelogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        homelogs[player  + '-' + formatted_date_string]['id'] = player
        homelogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days

https://statsapi.web.nhl.com/api/v1/game/2020020001/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020002/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020003/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020004/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020005/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020006/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020007/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020008/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020009/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020010/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020011/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020012/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020013/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020014/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020015/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020016/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2020020135/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020136/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020137/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020138/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020139/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020140/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020141/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020142/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020143/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020144/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020145/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020146/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020147/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020148/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020149/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020150/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2020020269/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020270/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020271/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020272/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020273/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020274/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020275/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020276/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020277/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020278/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020279/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020280/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020281/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020282/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020283/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020284/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2020020404/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020405/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020406/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020407/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020408/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020409/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020410/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020411/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020412/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020413/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020414/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020415/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020416/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020417/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020418/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020419/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2020020537/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020538/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020539/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020540/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020541/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020542/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020543/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020544/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020545/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020546/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020547/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020548/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020549/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020550/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020551/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020552/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2020020671/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020672/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020673/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020674/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020675/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020676/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020677/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020678/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020679/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020680/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020681/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020682/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020683/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020684/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020685/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020686/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2020020806/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020807/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020808/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020809/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020810/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020811/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020812/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020813/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020814/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020815/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020816/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020817/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020818/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020819/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020820/feed/live
https://statsapi.web.nhl.com/api/v1/game/2020020821/feed/live
https://

In [31]:
home_df = pd.DataFrame(homelogs)
home_df = home_df.transpose()

away_df = pd.DataFrame(awaylogs)
away_df = away_df.transpose()

columns_to_convert1 = ['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes']
columns_to_convert2 = ['faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists']
columns_to_convert3 = ['blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim', 'shutout']
columns_to_convert4 = ['saves', 'powerPlaySaves', 'shortHandedSaves', 'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst']

home_df[columns_to_convert1] = home_df[columns_to_convert1].fillna(0).astype(int)
home_df[columns_to_convert2] = home_df[columns_to_convert2].fillna(0).astype(int)
home_df[columns_to_convert3] = home_df[columns_to_convert3].fillna(0).astype(int)
home_df[columns_to_convert4] = home_df[columns_to_convert4].fillna(0).astype(int)

away_df[columns_to_convert1] = away_df[columns_to_convert1].fillna(0).astype(int)
away_df[columns_to_convert2] = away_df[columns_to_convert2].fillna(0).astype(int)
away_df[columns_to_convert3] = away_df[columns_to_convert3].fillna(0).astype(int)
away_df[columns_to_convert4] = away_df[columns_to_convert4].fillna(0).astype(int)

all_20_df = pd.concat([home_df, away_df], axis=0)

all_20_df['date'] = pd.to_datetime(all_20_df['date'], format='%d-%m-%Y')
all_20_df['birthdate'] = pd.to_datetime(all_20_df['birthdate'], format='%Y-%m-%d')
all_20_df = all_20_df.sort_values(by='date')

In [35]:
# 20-21 FANTASYPOINTS
for index, row in all_20_df.iterrows():
    d = 0
    so = 0
    w = 0

    if row['timeOnIce'] > 0:
        all_20_df.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
            all_20_df.at[index, 'shutout'] = 0
        if row['shutout'] == 1:
            so = 3
        else:
            so = 0
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
        all_20_df.at[index, 'shots_against'] = shots_against
        ga = (shots_against - row['saves']) * -2
        all_20_df.at[index, 'goals_against'] = abs(ga)/2
        saves = row['saves'] * .2
        fp = d + so + ga + saves
        all_20_df.at[index, 'fantasyPoints'] = fp
        all_20_df.at[index, 'decisionPoints'] = d
        all_20_df.at[index, 'wins'] = w

    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
        all_20_df.at[index, 'fantasyPoints'] = fp
        all_20_df.at[index, 'specialTeams'] = st


In [36]:
# Remove all zero ice time games

all_20_df = all_20_df[all_20_df['timeOnIce'] != 0]

In [37]:
# Replace Sebastian Aho, D, with Sebastian Aho (D)

all_20_df.loc[(all_20_df['name'] == 'Sebastian Aho') & (all_20_df['position'] == 'D')] = all_20_df.loc[(all_20_df['name'] == 'Sebastian Aho') & (all_20_df['position'] == 'D')].replace('Sebastian Aho', 'Sebastian Aho (D)')

In [42]:
all_20_df

,name,position,date,team,opponent,birthdate,id,aged,timeOnIce,assists,...,evenStrengthSavePercentage,shutout,shortHandedSavePercentage,fantasyPoints,specialTeams,gamePlayed,shots_against,goals_against,decisionPoints,wins
ID8475177-13-01-2021,Calvin de Haan,D,2021-01-13,CHI,TBL,1991-05-09,ID8475177,10842,1239,0,...,NaN,0,NaN,0.6,0.0,1.0,NaN,NaN,NaN,NaN
ID8478027-13-01-2021,Lucas Wallmark,C,2021-01-13,CHI,TBL,1995-09-05,ID8478027,9262,912,0,...,NaN,0,NaN,0.3,0.0,1.0,NaN,NaN,NaN,NaN
ID8470281-13-01-2021,Duncan Keith,D,2021-01-13,CHI,TBL,1983-07-16,ID8470281,13696,1183,0,...,NaN,0,NaN,1.7,0.0,1.0,NaN,NaN,NaN,NaN
ID8480459-13-01-2021,Pius Suter,C,2021-01-13,CHI,TBL,1996-05-24,ID8480459,9000,709,0,...,NaN,0,NaN,0.4,0.0,1.0,NaN,NaN,NaN,NaN
ID8474141-13-01-2021,Patrick Kane,RW,2021-01-13,CHI,TBL,1988-11-19,ID8474141,11743,1211,0,...,NaN,0,NaN,0.4,0.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8474574-19-05-2021,Tyler Myers,D,2021-05-19,VAN,CGY,1990-02-01,ID8474574,11430,1398,1,...,NaN,0,NaN,1.6,0.0,1.0,NaN,NaN,NaN,NaN
ID8477963-19-05-2021,Jayce Hawryluk,C,2021-05-19,VAN,CGY,1996-01-01,ID8477963,9270,417,0,...,NaN,0,NaN,0.2,0.0,1.0,NaN,NaN,NaN,NaN
ID8480800-19-05-2021,Quinn Hughes,D,2021-05-19,VAN,CGY,1999-10-14,ID8480800,7888,1373,0,...,NaN,0,NaN,0.6,0.0,1.0,NaN,NaN,NaN,NaN
ID8474612-19-05-2021,Travis Hamonic,D,2021-05-19,VAN,CGY,1990-08-16,ID8474612,11234,1084,0,...,NaN,0,NaN,0.7,0.0,1.0,NaN,NaN,NaN,NaN


In [66]:
all_20_df.to_csv('2020-21.csv', encoding='utf-8', index=False)

# Scrape for 2021-22

In [44]:

year = '202102'
date_format_game = "%d-%m-%Y"
date_format_birth = "%Y-%m-%d"

# pprint(game['liveData']['boxscore']['teams']['away']['players'])


awaylogs = {}
homelogs = {}


for i in range(1312):
# for i in range(2):    
    
    gameURL = baseURL + year + str(i+1).zfill(4) + appendix
    
    print(gameURL)

    game = requests.get(gameURL).json()
    
    away = game['liveData']['boxscore']['teams']['away']['players']
    awayname = game['gameData']['teams']['away']['abbreviation']
    home = game['liveData']['boxscore']['teams']['home']['players']
    homename = game['gameData']['teams']['home']['abbreviation']
    birthday = game['gameData']['players']

    if game['liveData']['linescore']['currentPeriod'] > 3:
        OT = True
    else:
        OT = False

    awayteam = []

    date = game['gameData']['datetime']['dateTime']

    datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

    utc_timestamp = pytz.utc.localize(datetime_obj)
    eastern_timezone = pytz.timezone('US/Eastern')
    eastern_time = utc_timestamp.astimezone(eastern_timezone)

    formatted_date_string = eastern_time.strftime('%d-%m-%Y')
    game_date = datetime.strptime(formatted_date_string, date_format)

    for player in away:
        awayteam.append(player)

    for player in awayteam:
        name = away[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = away[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = away[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = away[player]['position']['abbreviation']
            stats['name'] = name

        awaylogs[player  + '-' + formatted_date_string] = stats
        awaylogs[player  + '-' + formatted_date_string]['position'] = position
        awaylogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        awaylogs[player  + '-' + formatted_date_string]['name'] = name
        awaylogs[player  + '-' + formatted_date_string]['team'] = awayname
        awaylogs[player  + '-' + formatted_date_string]['opponent'] = homename
        awaylogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        awaylogs[player  + '-' + formatted_date_string]['id'] = player
        awaylogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days


    hometeam = []

    # date = game['gameData']['datetime']['dateTime']

    for player in home:
        hometeam.append(player)

    for player in hometeam:
        name = home[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = home[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = home[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = home[player]['position']['abbreviation']
            stats['name'] = name
            
                

        homelogs[player  + '-' + formatted_date_string] = stats
        homelogs[player  + '-' + formatted_date_string]['position'] = position
        homelogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        homelogs[player  + '-' + formatted_date_string]['name'] = name
        homelogs[player  + '-' + formatted_date_string]['team'] = homename
        homelogs[player  + '-' + formatted_date_string]['opponent'] = awayname
        homelogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        homelogs[player  + '-' + formatted_date_string]['id'] = player
        homelogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days

https://statsapi.web.nhl.com/api/v1/game/2021020001/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020002/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020003/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020004/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020005/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020006/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020007/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020008/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020009/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020010/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020011/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020012/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020013/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020014/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020015/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020016/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2021020135/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020136/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020137/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020138/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020139/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020140/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020141/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020142/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020143/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020144/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020145/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020146/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020147/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020148/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020149/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020150/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2021020268/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020269/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020270/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020271/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020272/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020273/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020274/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020275/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020276/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020277/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020278/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020279/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020280/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020281/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020282/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020283/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2021020402/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020403/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020404/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020405/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020406/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020407/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020408/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020409/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020410/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020411/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020412/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020413/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020414/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020415/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020416/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020417/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2021020535/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020536/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020537/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020538/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020539/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020540/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020541/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020542/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020543/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020544/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020545/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020546/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020547/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020548/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020549/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020550/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2021020668/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020669/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020670/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020671/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020672/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020673/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020674/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020675/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020676/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020677/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020678/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020679/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020680/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020681/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020682/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020683/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2021020801/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020802/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020803/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020804/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020805/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020806/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020807/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020808/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020809/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020810/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020811/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020812/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020813/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020814/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020815/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020816/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2021020934/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020935/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020936/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020937/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020938/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020939/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020940/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020941/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020942/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020943/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020944/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020945/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020946/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020947/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020948/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021020949/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2021021067/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021068/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021069/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021070/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021071/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021072/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021073/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021074/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021075/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021076/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021077/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021078/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021079/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021080/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021081/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021082/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2021021201/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021202/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021203/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021204/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021205/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021206/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021207/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021208/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021209/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021210/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021211/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021212/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021213/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021214/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021215/feed/live
https://statsapi.web.nhl.com/api/v1/game/2021021216/feed/live
https://

In [45]:
home_df = pd.DataFrame(homelogs)
home_df = home_df.transpose()

away_df = pd.DataFrame(awaylogs)
away_df = away_df.transpose()

columns_to_convert1 = ['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes']
columns_to_convert2 = ['faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists']
columns_to_convert3 = ['blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim', 'shutout']
columns_to_convert4 = ['saves', 'powerPlaySaves', 'shortHandedSaves', 'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst']

home_df[columns_to_convert1] = home_df[columns_to_convert1].fillna(0).astype(int)
home_df[columns_to_convert2] = home_df[columns_to_convert2].fillna(0).astype(int)
home_df[columns_to_convert3] = home_df[columns_to_convert3].fillna(0).astype(int)
home_df[columns_to_convert4] = home_df[columns_to_convert4].fillna(0).astype(int)

away_df[columns_to_convert1] = away_df[columns_to_convert1].fillna(0).astype(int)
away_df[columns_to_convert2] = away_df[columns_to_convert2].fillna(0).astype(int)
away_df[columns_to_convert3] = away_df[columns_to_convert3].fillna(0).astype(int)
away_df[columns_to_convert4] = away_df[columns_to_convert4].fillna(0).astype(int)

all_21_df = pd.concat([home_df, away_df], axis=0)

all_21_df['date'] = pd.to_datetime(all_21_df['date'], format='%d-%m-%Y')
all_21_df['birthdate'] = pd.to_datetime(all_21_df['birthdate'], format='%Y-%m-%d')
all_21_df = all_21_df.sort_values(by='date')

In [46]:
# 21-22 FANTASYPOINTS
for index, row in all_21_df.iterrows():
    d = 0
    so = 0
    w = 0

    if row['timeOnIce'] > 0:
        all_21_df.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
            all_21_df.at[index, 'shutout'] = 0
        if row['shutout'] == 1:
            so = 3
        else:
            so = 0
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
        all_21_df.at[index, 'shots_against'] = shots_against
        ga = (shots_against - row['saves']) * -2
        all_21_df.at[index, 'goals_against'] = abs(ga)/2
        saves = row['saves'] * .2
        fp = d + so + ga + saves
        all_21_df.at[index, 'fantasyPoints'] = fp
        all_21_df.at[index, 'decisionPoints'] = d
        all_21_df.at[index, 'wins'] = w

    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
        all_21_df.at[index, 'fantasyPoints'] = fp
        all_21_df.at[index, 'specialTeams'] = st


In [47]:
# Remove all zero ice time games

all_21_df = all_21_df[all_21_df['timeOnIce'] != 0]

In [48]:
# Replace Sebastian Aho, D, with Sebastian Aho (D)

all_21_df.loc[(all_21_df['name'] == 'Sebastian Aho') & (all_21_df['position'] == 'D')] = all_21_df.loc[(all_21_df['name'] == 'Sebastian Aho') & (all_21_df['position'] == 'D')].replace('Sebastian Aho', 'Sebastian Aho (D)')

In [51]:
all_21_df

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,...,shutout,powerPlaySavePercentage,shortHandedSavePercentage,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8480172-12-10-2021,955,0,0,3,0,0,0,0,0,0,...,0,NaN,NaN,1.0,0.8,0.0,NaN,NaN,NaN,NaN
ID8478046-12-10-2021,1111,0,1,4,2,0,0,2,0,0,...,0,NaN,NaN,1.0,3.6,0.0,NaN,NaN,NaN,NaN
ID8478542-12-10-2021,1005,0,1,3,2,0,0,0,7,11,...,0,NaN,NaN,1.0,2.5,0.0,NaN,NaN,NaN,NaN
ID8477244-12-10-2021,949,0,0,1,1,0,0,0,0,0,...,0,NaN,NaN,1.0,0.7,0.0,NaN,NaN,NaN,NaN
ID8478017-12-10-2021,847,0,0,2,2,0,0,0,0,0,...,0,NaN,NaN,1.0,1.4,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8477940-01-05-2022,914,0,0,1,0,0,0,0,0,0,...,0,NaN,NaN,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8478424-01-05-2022,765,2,0,0,0,0,0,0,3,3,...,0,NaN,NaN,1.0,2.0,0.0,NaN,NaN,NaN,NaN
ID8475455-01-05-2022,1022,0,0,0,4,0,0,0,0,0,...,0,NaN,NaN,1.0,1.4,0.0,NaN,NaN,NaN,NaN
ID8478431-01-05-2022,793,0,0,1,1,0,0,0,0,0,...,0,NaN,NaN,1.0,0.7,0.0,NaN,NaN,NaN,NaN


In [65]:
all_21_df.to_csv('2021-22.csv', encoding='utf-8', index=False)

# Scrape 2022-23

In [53]:

year = '202202'
date_format_game = "%d-%m-%Y"
date_format_birth = "%Y-%m-%d"

# pprint(game['liveData']['boxscore']['teams']['away']['players'])


awaylogs = {}
homelogs = {}


for i in range(1312):
# for i in range(2):    
    
    gameURL = baseURL + year + str(i+1).zfill(4) + appendix
    
    print(gameURL)

    game = requests.get(gameURL).json()
    
    away = game['liveData']['boxscore']['teams']['away']['players']
    awayname = game['gameData']['teams']['away']['abbreviation']
    home = game['liveData']['boxscore']['teams']['home']['players']
    homename = game['gameData']['teams']['home']['abbreviation']
    birthday = game['gameData']['players']

    if game['liveData']['linescore']['currentPeriod'] > 3:
        OT = True
    else:
        OT = False

    awayteam = []

    date = game['gameData']['datetime']['dateTime']

    datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

    utc_timestamp = pytz.utc.localize(datetime_obj)
    eastern_timezone = pytz.timezone('US/Eastern')
    eastern_time = utc_timestamp.astimezone(eastern_timezone)

    formatted_date_string = eastern_time.strftime('%d-%m-%Y')
    game_date = datetime.strptime(formatted_date_string, date_format)

    for player in away:
        awayteam.append(player)

    for player in awayteam:
        name = away[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = away[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = away[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = away[player]['position']['abbreviation']
            stats['name'] = name

        awaylogs[player  + '-' + formatted_date_string] = stats
        awaylogs[player  + '-' + formatted_date_string]['position'] = position
        awaylogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        awaylogs[player  + '-' + formatted_date_string]['name'] = name
        awaylogs[player  + '-' + formatted_date_string]['team'] = awayname
        awaylogs[player  + '-' + formatted_date_string]['opponent'] = homename
        awaylogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        awaylogs[player  + '-' + formatted_date_string]['id'] = player
        awaylogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days


    hometeam = []

    # date = game['gameData']['datetime']['dateTime']

    for player in home:
        hometeam.append(player)

    for player in hometeam:
        name = home[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = home[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = home[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = home[player]['position']['abbreviation']
            stats['name'] = name
            
                

        homelogs[player  + '-' + formatted_date_string] = stats
        homelogs[player  + '-' + formatted_date_string]['position'] = position
        homelogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        homelogs[player  + '-' + formatted_date_string]['name'] = name
        homelogs[player  + '-' + formatted_date_string]['team'] = homename
        homelogs[player  + '-' + formatted_date_string]['opponent'] = awayname
        homelogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        homelogs[player  + '-' + formatted_date_string]['id'] = player
        homelogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days

https://statsapi.web.nhl.com/api/v1/game/2022020001/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020002/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020003/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020004/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020005/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020006/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020007/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020008/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020009/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020010/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020011/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020012/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020013/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020014/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020015/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020016/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2022020134/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020135/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020136/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020137/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020138/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020139/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020140/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020141/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020142/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020143/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020144/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020145/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020146/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020147/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020148/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020149/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2022020267/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020268/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020269/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020270/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020271/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020272/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020273/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020274/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020275/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020276/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020277/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020278/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020279/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020280/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020281/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020282/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2022020400/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020401/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020402/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020403/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020404/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020405/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020406/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020407/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020408/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020409/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020410/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020411/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020412/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020413/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020414/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020415/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2022020535/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020536/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020537/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020538/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020539/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020540/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020541/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020542/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020543/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020544/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020545/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020546/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020547/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020548/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020549/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020550/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2022020668/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020669/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020670/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020671/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020672/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020673/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020674/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020675/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020676/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020677/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020678/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020679/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020680/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020681/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020682/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020683/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2022020801/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020802/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020803/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020804/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020805/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020806/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020807/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020808/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020809/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020810/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020811/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020812/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020813/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020814/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020815/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020816/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2022020935/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020936/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020937/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020938/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020939/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020940/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020941/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020942/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020943/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020944/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020945/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020946/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020947/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020948/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020949/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022020950/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2022021070/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021071/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021072/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021073/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021074/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021075/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021076/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021077/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021078/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021079/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021080/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021081/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021082/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021083/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021084/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021085/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2022021203/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021204/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021205/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021206/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021207/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021208/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021209/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021210/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021211/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021212/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021213/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021214/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021215/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021216/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021217/feed/live
https://statsapi.web.nhl.com/api/v1/game/2022021218/feed/live
https://

In [56]:
home_df = pd.DataFrame(homelogs)
home_df = home_df.transpose()

away_df = pd.DataFrame(awaylogs)
away_df = away_df.transpose()

columns_to_convert1 = ['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes']
columns_to_convert2 = ['faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists']
columns_to_convert3 = ['blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim', 'shutout']
columns_to_convert4 = ['saves', 'powerPlaySaves', 'shortHandedSaves', 'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst']

home_df[columns_to_convert1] = home_df[columns_to_convert1].fillna(0).astype(int)
home_df[columns_to_convert2] = home_df[columns_to_convert2].fillna(0).astype(int)
home_df[columns_to_convert3] = home_df[columns_to_convert3].fillna(0).astype(int)
home_df[columns_to_convert4] = home_df[columns_to_convert4].fillna(0).astype(int)

away_df[columns_to_convert1] = away_df[columns_to_convert1].fillna(0).astype(int)
away_df[columns_to_convert2] = away_df[columns_to_convert2].fillna(0).astype(int)
away_df[columns_to_convert3] = away_df[columns_to_convert3].fillna(0).astype(int)
away_df[columns_to_convert4] = away_df[columns_to_convert4].fillna(0).astype(int)

all_22_df = pd.concat([home_df, away_df], axis=0)

all_22_df['date'] = pd.to_datetime(all_22_df['date'], format='%d-%m-%Y')
all_22_df['birthdate'] = pd.to_datetime(all_22_df['birthdate'], format='%Y-%m-%d')
all_22_df = all_22_df.sort_values(by='date')

In [58]:
# 22-23 FANTASYPOINTS
for index, row in all_22_df.iterrows():
    d = 0
    so = 0
    w = 0

    if row['timeOnIce'] > 0:
        all_22_df.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
            all_22_df.at[index, 'shutout'] = 0
        if row['shutout'] == 1:
            so = 3
        else:
            so = 0
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
        all_22_df.at[index, 'shots_against'] = shots_against
        ga = (shots_against - row['saves']) * -2
        all_22_df.at[index, 'goals_against'] = abs(ga)/2
        saves = row['saves'] * .2
        fp = d + so + ga + saves
        all_22_df.at[index, 'fantasyPoints'] = fp
        all_22_df.at[index, 'decisionPoints'] = d
        all_22_df.at[index, 'wins'] = w

    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
        all_22_df.at[index, 'fantasyPoints'] = fp
        all_22_df.at[index, 'specialTeams'] = st


In [59]:
# Remove all zero ice time games

all_22_df = all_22_df[all_22_df['timeOnIce'] != 0]

In [60]:
# Replace Sebastian Aho, D, with Sebastian Aho (D)

all_22_df.loc[(all_22_df['name'] == 'Sebastian Aho') & (all_22_df['position'] == 'D')] = all_22_df.loc[(all_22_df['name'] == 'Sebastian Aho') & (all_22_df['position'] == 'D')].replace('Sebastian Aho', 'Sebastian Aho (D)')

In [63]:
# Replace Matt Murray, DAL, with Matt Murray (DAL)

all_22_df.loc[(all_22_df['name'] == 'Matt Murray') & (all_22_df['team'] == 'DAL')] = all_22_df.loc[(all_22_df['name'] == 'Matt Murray') & (all_22_df['team'] == 'DAL')].replace('Matt Murray', 'Matt Murray (DAL)')

In [64]:
all_22_df.to_csv('2022-23.csv', encoding='utf-8', index=False)